## Legiscan Summary Scraper

In [31]:
# IMPORT PACKAGES
from selenium import webdriver         
from selenium.webdriver.common.by import By   
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time                                   
from bs4 import BeautifulSoup                 
import pandas as pd                             
import re                                       
import numpy as np
import requests
import io
import os

In [32]:
# READ IN BILL INFO
bill_data = pd.read_csv("../modified_data/merged_bill_data.csv")
# Make a list of the bill names
names_list = bill_data.bill_name
states_list = bill_data.state

In [33]:
summaries = []
# Start a new Chrome web driver
driver = webdriver.Chrome(ChromeDriverManager().install())
# Go to legiscan website
driver.get("https://legiscan.com/AL/legislation")
for i in range(len(names_list)):
    print("BILL:", states_list[i], names_list[i])
    # Wait
    driver.implicitly_wait(3)
    # Select state from dropdown
    dropdown = driver.find_element(By.ID, "edit-state-id")
    select = Select(dropdown)
    select.select_by_visible_text(states_list[i])
    # Try to find the bill number box and clear it 
    try:
        driver.find_element(By.ID, "edit-bill-number").clear()
    # If the code runs with error
    except:
        # Try to find the bill number box another way
        try:
            driver.find_element(By.ID, "edit-bill-number-1").clear()
        # If none of this works, print an error statement
        except: 
            print("ERROR Not able to edit the bill number for:", states_list[i], names_list[i])
        # If the other way works, clear the box and send the keys
        else:
            driver.find_element(By.ID, "edit-bill-number-1").clear()
            input_box = driver.find_element(By.ID, "edit-bill-number-1")
            input_box.send_keys(names_list[i])
            # Click search
            driver.find_element(By.ID, "edit-submit").click()
            # Try to find the bill summary
            try: 
                driver.find_element(By.ID, "bill-summary")
            # If no bill summary found, fill with NaN and print error message
            except: 
                print("ERROR Could not find bill summary for:", states_list[i], names_list[i])
                summaries.append(np.nan)
            # If bill summary is found, add to list
            else:
                summary_element = driver.find_element(By.ID, "bill-summary")
                summaries.append(summary_element.text)
    # If the first way of finding the box with the bill number works, run the following
    else:
        # Clear box and send bill number to box
        driver.find_element(By.ID, "edit-bill-number").clear()
        input_box = driver.find_element(By.ID, "edit-bill-number")
        input_box.send_keys(names_list[i])
        # Click search
        driver.find_element(By.ID, "edit-submit").click()
        # Try to find the bill summary
        try: 
            driver.find_element(By.ID, "bill-summary")
        # If no bill summary found, fill with NaN and print error message
        except: 
            print("ERROR Could not find bill summary for:", states_list[i], names_list[i])
            summaries.append(np.nan)
        # If bill summary is found, add to list
        else:
            summary_element = driver.find_element(By.ID, "bill-summary")
            summaries.append(summary_element.text)
driver.quit()

WebDriverException: Message: unknown error: net::ERR_CONNECTION_REFUSED
  (Session info: chrome=111.0.5563.146)
